In [ ]:
def Rho_Less_Computation(P,Q,s):
    '''
    Función que resuelve el problema del logaritmo discreto en una curva elíptica usando una versión del algoritmo Rho Pollard 
    en el que se tarda menos tiempo en ejecutarlo pero se necesita más memoria.
    Entrada: dos puntos P y Q de una curva elíptica definida en un cuerpo finito y un número s que definirá en cuantos grupos 
    se divide la función f.
    Salida: un número k que cumple que k*P = Q.
    '''
    N = P.order()
    d = N
    
    while d == N:
        
        #Construcción de todos los elementos de la función f
        M = []
        for i in range(s):
            A = [randint(2,N), randint(2,N),0]
            A = [A[0], A[1], A[0]*P + A[1]*Q] #Estructura formada por los índices de cada punto con 
                                                #respecto a P y Q y dicho punto
            M.append(A)
        
        listInd = [] #Lista de índices de como se han formado los puntos con respecto a P y Q
        listPun = [] #Lista de puntos formados con los indices anteriores
        P0Ind = [randint(2,N), randint(2,N)] #Índices del punto inicial
        P0Pun = P0Ind[0]*P + P0Ind[1]*Q #Punto inicial
        listInd.append(P0Ind)
        listPun.append(P0Pun)
        
        #Se genera el siguiente punto con la función f dependiendo de la coordenada x del punto en módulo s
        i = int(P0Pun[0])%s
        UInd = [P0Ind[0] + M[i][0], P0Ind[1] + M[i][1]]
        UPun = P0Pun + M[i][2]
        
        #Se busca la colisión
        while UPun not in listPun:
            listInd.append(UInd)
            listPun.append(UPun)
            i = int(UPun[0])%s
            UInd = [UInd[0] + M[i][0], UInd[1] + M[i][1]]
            UPun = UPun + M[i][2]
        
        #Guardamos los indices del otro punto que colisiona
        V = listInd[listPun.index(UPun)]
        a = UInd[1] - V[1]
        b = V[0] - UInd[0]
        d = gcd(a,N)
    
    n = N/d
    dif = gcd(a,n)
    
    #Nos aseguramos de que existe inverso en el módulo n
    while dif != 1:
        n = n/dif
        dif = gcd(a,n)
    n = int(n)
    inv = inverse_mod(a,n)
    k = inv*b
    k = k%n
    j = 0
    
    #En caso de que d > 1, comprobamos cual de los posibles valores para k es el correcto
    while j < d:
        if k*P == Q:
            break
        else:
            k = k + n
            j = j + 1
    return k

In [ ]:
import numpy as np
i = next_prime(1000)
cuerpos = []
ordenes = []
tiempos_medias = []
tiempos_desviaciones = []
while i < 30000:
    E = EllipticCurve(GF(i),[2,1])
    N = E.order() - 1
    j = 0
    times = []
    while j < 10:
        P = E[randint(1,N)]
        n = randint(1,N)
        Q = n*P
        time_Inicial = cputime()
        k = Rho_Less_Computation(P,Q,20)
        time_Final = cputime() - time_Inicial
        times.append(time_Final)
        j = j + 1
    tiempos_medias.append(np.array(times).mean())
    tiempos_desviaciones.append(np.array(times).std())
    ordenes.append(N)
    cuerpos.append(i)
    i = next_prime(i)
import csv
with open('Datos_Rho_Less_Computation.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    k = 0
    while k < len(ordenes):
        writer.writerow([str(cuerpos[k])]+[str(ordenes[k])]+[str(tiempos_medias[k])]+[str(tiempos_desviaciones[k])])
        k = k + 1

In [ ]:
import csv
Datos1 = []
Datos2 = []
with open('Datos_Rho_Less_Computation.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        Datos1.append(int(row[0]))
        Datos2.append(float(row[2]))
recta = line(zip(Datos1,Datos2),axes_labels=['$Orden$ $Cuerpo$ $Finito$','$Tiempo$ $Medio$ $(s)$'],frame=True)
recta.show()

In [ ]:
import csv
Datos1 = []
Datos2 = []
with open('Datos_Rho_Less_Computation.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        Datos1.append(int(row[0]))
        Datos2.append(float(row[3]))
recta = line(zip(Datos1,Datos2),axes_labels=['$Orden$ $Cuerpo$ $Finito$','$Desviación$'],frame=True)
recta.show()